## SBIR Company Dedupe with Patent Organization Data

In [1]:
import pandas as pd
import numpy as np 
import dedupe
import os

In [2]:
df_patent_org = pd.read_csv('patent_org_2017.csv')

In [24]:
cols = ['Date', 'Header', 'Abstract', 'Clean_Funding', 'Topics']
df_sb = pd.read_csv('sbir_award_data.csv')

/Users/ifti/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (10,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_sb.columns

Index(['Company', 'Award Title', 'Agency', 'Branch', 'Phase', 'Program',
       'Agency Tracking Number', 'Contract', 'Proposal Award Date',
       'Contract End Date', 'Solicitation Number', 'Solicitation Year',
       'Topic Code', 'Award Year', 'Award Amount', 'Duns', 'HUBZone Owned',
       'Socially and Economically Disadvantaged', 'Women Owned',
       'Number Employees', 'Company Website', 'Address1', 'Address2', 'City',
       'State', 'Zip', 'Abstract', 'Contact Name', 'Contact Title',
       'Contact Phone', 'Contact Email', 'PI Name', 'PI Title', 'PI Phone',
       'PI Email', 'RI Name', 'RI POC Name', 'RI POC Phone'],
      dtype='object')

In [6]:
df_patent_org

,assignee_id,organization
0,3b86ee5d-49e0-444b-800a-67eb3c0b4f2c,ITSAUL PLANTS LLC
1,ed25dd8e-2465-47f2-b1e7-00d223089af4,Altman Plants
2,51a6da5b-4755-4e2f-a282-159552a45a42,"Georgia State University Research Foundation, ..."
3,37d3b9fd-1d06-4d7a-a5dd-1f9334bbd292,SYNGENTA PARTICIPATIONS AG
4,6cf8e9cb-e280-4ef2-a0fd-a371b7a33a12,Danziger “DAN” Flower Farm
...,...,...
45745,05d390a7-27aa-4758-8355-f53154d7df0f,"lonis Pharmaceuticals, Inc."
45746,56d0d9a4-5315-4e34-97ea-d8afe96e6ac6,"Aflnitl, Ltd."
45747,dd4fa4c6-2282-47d5-85f9-41994c3f9a33,"ALGAE TO OMEGA HOLDINGS, INC."
45748,707b39e5-8ea2-4a1a-a0c5-1996442181d4,"Panasonic Intellectual Property Mangement Co.,..."


In [7]:
df_patent_org['Company'] = df_patent_org['organization']

In [25]:
df_sb['Company'] = df_sb['Company'].astype(str)

In [14]:
data1_dict = df_patent_org.to_dict('index')
data2_dict = df_sb.to_dict('index')

In [15]:
output_file = 'data_matching_output.csv'
settings_file = 'data_matching_learned_settings'
training_file = 'data_matching_training.json'

if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as sf:
        linker = dedupe.StaticRecordLink(sf)

else:
    # Define the fields the linker will pay attention to

    fields = [
        {'field': 'Company', 'type': 'String'}
        # , {'field': 'Company', 'type': 'Text' , 'corpus': descriptions()}
            ]

    # Create a new linker object and pass our data model to it.
    linker = dedupe.RecordLink(fields)

    # If we have training data saved from a previous run of linker,
    # look for it an load it in.
    # __Note:__ if you want to train from scratch, delete the training_file

  # Create a new linker object and pass our data model to it.
    linker = dedupe.RecordLink(fields)

    # If we have training data saved from a previous run of linker,
    # look for it an load it in.
    # __Note:__ if you want to train from scratch, delete the training_file
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file) as tf:
            linker.prepare_training(data1_dict,
                                    data2_dict,
                                    training_file=tf,
                                    sample_size=1500)
    else:
        linker.prepare_training(data1_dict, data2_dict, sample_size=1500)

    # ## Active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as matches
    # or not.
    # use 'y', 'n' and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')

    dedupe.console_label(linker)

    linker.train()

    # When finished, save our training away to disk
    with open(training_file, 'w') as tf:
        linker.write_training(tf)

    # Save our weights and predicates to disk.  If the settings file
    # exists, we will skip all the training and learning next time we run
    # this file.
    with open(settings_file, 'wb') as sf:
        linker.write_settings(sf)


INFO:dedupe.canopy_index:Removing stop word  I
INFO:dedupe.canopy_index:Removing stop word In
INFO:dedupe.canopy_index:Removing stop word al
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word ic
INFO:dedupe.canopy_index:Removing stop word nc
INFO:dedupe.canopy_index:Removing stop word ra
INFO:dedupe.canopy_index:Removing stop word st
INFO:dedupe.canopy_index:Removing stop word e 
INFO:dedupe.canopy_index:Removing stop word l 
INFO:dedupe.canopy_index:Removing stop word ri
INFO:dedupe.canopy_index:Removing stop word  L
INFO:dedupe.canopy_index:Removing stop word EC
INFO:dedupe.canopy_index:Removing stop word TE
INFO:dedupe.canopy_index:Removing stop word ER
INFO:dedupe.canopy_index:Removing stop word RA
INFO:dedupe.canopy_index:Removing stop word AT
INFO:dedupe.canopy_index:Removing stop word IN
INFO:dedupe.canopy_index:Removing stop word NC
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word ce
INFO:dedupe.c

starting active labeling...


 y


Company : Cybernet Systems Corp.

Company : Cybernet Systems Corporation

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 y


...

Company : New Dimension One Spas, Inc.

Company : New Dimension Research

10/10 positive, 11/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 n


Company : PROTON ENERGY SYSTEMS, INC.

Company : ITN ENERGY SYSTEMS, INC.

10/10 positive, 12/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 f


Finished labeling
INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.100000, score 0.13079668297139563
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameSevenCharStartPredicate, Company), SimplePredicate: (commonFourGram, Company), SimplePredicate: (commonTwoTokens, Company))
INFO:dedupe.training:(SimplePredicate: (oneGramFingerprint, Company), TfidfNGramSearchPredicate: (0.2, Company), LevenshteinSearchPredicate: (2, Company))


In [16]:
# ## Blocking

# ## Clustering

# Find the threshold that will maximize a weighted average of our
# precision and recall.  When we set the recall weight to 2, we are
# saying we care twice as much about recall as we do precision.
#
# If we had more data, we would not pass in all the blocked data into
# this function but a representative sample.

print('clustering...')
linked_records = linker.join(data1_dict, data2_dict, 0.0)

print('# duplicate sets', len(linked_records))
# ## Writing Results

# Write our original data back out to a CSV with a new column called
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership2 = {}
for cluster_id, (cluster, score) in enumerate(linked_records):
    for record_id in cluster:
        cluster_membership2[record_id] = {'Cluster ID': cluster_id,
                                         'Link Score': score}
print(cluster_membership2)


clustering...


INFO:dedupe.canopy_index:Removing stop word  I
INFO:dedupe.canopy_index:Removing stop word In
INFO:dedupe.canopy_index:Removing stop word nc
INFO:dedupe.canopy_index:Removing stop word  L
INFO:dedupe.canopy_index:Removing stop word EC
INFO:dedupe.canopy_index:Removing stop word TE
INFO:dedupe.canopy_index:Removing stop word LC
INFO:dedupe.canopy_index:Removing stop word ch
INFO:dedupe.canopy_index:Removing stop word h 
INFO:dedupe.canopy_index:Removing stop word st
INFO:dedupe.canopy_index:Removing stop word ER
INFO:dedupe.canopy_index:Removing stop word RA
INFO:dedupe.canopy_index:Removing stop word ro
INFO:dedupe.canopy_index:Removing stop word IN
INFO:dedupe.canopy_index:Removing stop word AT
INFO:dedupe.canopy_index:Removing stop word ES
INFO:dedupe.canopy_index:Removing stop word NO
INFO:dedupe.canopy_index:Removing stop word ON
INFO:dedupe.canopy_index:Removing stop word S 
INFO:dedupe.canopy_index:Removing stop word TI
INFO:dedupe.canopy_index:Removing stop word  C
INFO:dedupe.c

# duplicate sets 1247
{35071: {'Cluster ID': 85, 'Link Score': 0.65499824}, 42495: {'Cluster ID': 0, 'Link Score': 0.65499824}, 5375: {'Cluster ID': 1, 'Link Score': 0.65499824}, 118068: {'Cluster ID': 1, 'Link Score': 0.65499824}, 13310: {'Cluster ID': 2, 'Link Score': 0.65499824}, 43518: {'Cluster ID': 2, 'Link Score': 0.65499824}, 5117: {'Cluster ID': 3, 'Link Score': 0.65499824}, 2783: {'Cluster ID': 3, 'Link Score': 0.65499824}, 43772: {'Cluster ID': 4, 'Link Score': 0.65499824}, 5875: {'Cluster ID': 4, 'Link Score': 0.65499824}, 23292: {'Cluster ID': 5, 'Link Score': 0.65499824}, 150780: {'Cluster ID': 5, 'Link Score': 0.65499824}, 42235: {'Cluster ID': 6, 'Link Score': 0.65499824}, 24503: {'Cluster ID': 6, 'Link Score': 0.65499824}, 40187: {'Cluster ID': 7, 'Link Score': 0.65499824}, 29741: {'Cluster ID': 7, 'Link Score': 0.65499824}, 35067: {'Cluster ID': 8, 'Link Score': 0.65499824}, 78797: {'Cluster ID': 8, 'Link Score': 0.65499824}, 25339: {'Cluster ID': 9, 'Link Score': 0.6

In [17]:
len(cluster_membership2)

2482

In [19]:
cluster_d = {}
i = 0
for k, v in cluster_membership2.items():
#     print(k,v['Cluster ID'])
    cluster_id = v['Cluster ID']
    keys = cluster_d.get(cluster_id, [])
    keys.append(k)
    cluster_d[cluster_id] = keys
    i+=1
    if i >=2000:
        break
# print(cluster_d)

In [27]:
df_sb['assignee_id'] = ""
df_sb['organization'] = ""
for k, v in cluster_d.items():
#     print(k)
#     print(df_patent_org.iloc[v[0]]['assignee_id'])
    try:
        df_sb.at[v[1], 'assignee_id'] = df_patent_org.iloc[v[0]]['assignee_id']
        df_sb.at[v[1], 'organization'] = df_patent_org.iloc[v[0]]['organization']
    except:
        pass


In [22]:
df_sb.columns

Index(['Company', 'Award Title', 'Agency', 'Branch', 'Phase', 'Program',
       'Agency Tracking Number', 'Contract', 'Proposal Award Date',
       'Contract End Date', 'Solicitation Number', 'Solicitation Year',
       'Topic Code', 'Award Year', 'Award Amount', 'Duns', 'HUBZone Owned',
       'Socially and Economically Disadvantaged', 'Women Owned',
       'Number Employees', 'Company Website', 'Address1', 'Address2', 'City',
       'State', 'Zip', 'Abstract', 'Contact Name', 'Contact Title',
       'Contact Phone', 'Contact Email', 'PI Name', 'PI Title', 'PI Phone',
       'PI Email', 'RI Name', 'RI POC Name', 'RI POC Phone', 'assignee_id',
       'organization'],
      dtype='object')

In [32]:
# df_sb.dropna(inplace=True)
df_sb_patent = df_sb[df_sb['assignee_id'] != ""]

In [34]:
cols = ['Company','organization','assignee_id','Award Amount','Socially and Economically Disadvantaged', 'Women Owned','State']
df_sb_patent[cols]

,Company,organization,assignee_id,Award Amount,Socially and Economically Disadvantaged,Women Owned,State
119,"Aura Intelligent Systems, Inc.","Aura Intelligent Systems, Inc.",b3b07262-1aa1-4cff-b14c-4ecbf9995a86,"256,000",N,Y,MA
232,"Chiral Photonics, Inc.","Chiral Photonics, Inc.",1255db6b-aa99-43bf-b63b-bb1f3f18b18f,"1,125,745",N,N,NJ
401,Farad Power Inc,"Farad Power, Inc",42cf33c3-6c26-4fa8-a326-e580c019ce46,"132,383",N,N,CA
444,"Gate Scientific, Inc.","Gate Scientific, Inc.",46bb2e8b-7bd4-45eb-862d-55d097e5d1fa,"999,656",N,N,CA
482,"H3D, Inc.","H3D, Inc.",203dee44-5faa-4bae-b2ab-b12e492e9177,"200,000",N,N,MI
...,...,...,...,...,...,...,...
187875,Plasma Tech Corp.,"Plasma Tech Holdings, LLC",ee79c702-d68d-48d7-86c1-f59744dd4afd,"41,415",N,N,IL
188104,"Synergetics, Inc.","Synergetics, Inc.",80c21baa-31ec-4bf9-ae87-962881913fcb,"371,845",N,N,NC
188557,Environmental Protection,"Environmental Protection Administration, R.O.C.",ab221a9c-1281-4a8e-ac7c-478f3dd18f73,"50,000",N,N,CA
188598,General Technology Inc,Generalplus Technology Inc.,0de653f4-d162-4cfc-890b-503c9623e346,"49,607",N,N,VA


In [35]:
df_sb_patent.to_csv('df_sb_patent.csv', columns=cols, index=False)